## UTKFace Experiments (Black-Box PIA)
### Load UTKFace dataset

In [35]:
from random import shuffle
import os
from keras.utils import load_img, img_to_array, to_categorical
from keras.applications.mobilenet import preprocess_input


def get_gender_classes(filelist):
    gender = [i.split('_')[1] for i in filelist]

    result = []
    for i in gender:
        i = int(i)
        if i == 0:
            result.append(0)
        elif i == 1:
            result.append(1)
    return result


def get_utkface_gender_prediction_dataset(files_dir):
    onlyfiles = os.listdir(files_dir)
    #if do_shuffle:
    #    shuffle(onlyfiles)

    gender_classes = get_gender_classes(onlyfiles)

    # convert images to vectors
    X = []
    for file in onlyfiles:
        image_path = os.path.join(files_dir, file)
        rgb_image = load_img(image_path, target_size=(64, 64))
        x = img_to_array(rgb_image)
        #x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X.append(x)


    # normalize data
    #X = X.astype('float32')
    #X /= 255

    Y = to_categorical(gender_classes, num_classes=2)
    X = np.asarray(X)
    
    return X, Y, onlyfiles


In [36]:
utk_root = 'data/utkface'
X, Y, onlyfiles = get_utkface_gender_prediction_dataset(utk_root)

In [37]:
X[0].shape

(64, 64, 3)

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### MobileNet for UTKFace

In [39]:
from keras.applications import MobileNetV3Small
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam, AdamW, RMSprop, SGD
from keras.losses import CategoricalCrossentropy

mobilenet_mdl = MobileNetV3Small(
    include_top=False,
    input_shape=(64, 64, 3),
    weights="imagenet",
    alpha=0.75
)

In [40]:
def head_model(base_model, n_classes):
    top_model = base_model.output
    top_model = GlobalAveragePooling2D(data_format="channels_last")(top_model)
    top_model = Dense(1024, activation="relu")(top_model)
    top_model = Dropout(0.69)(top_model)
    top_model = Dense(1000, activation="relu")(top_model)
    top_model = Dropout(0.55)(top_model)
    top_model = Dense(n_classes, activation="softmax")(top_model)
    return top_model

In [41]:
model_mobilenet_head = head_model(mobilenet_mdl, len(Y[0]))
model_mobilenet = Model(inputs = mobilenet_mdl.input, outputs = model_mobilenet_head)

In [42]:
for layer in model_mobilenet.layers[:-6]:
    layer.trainable = False

In [43]:
model_mobilenet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 64, 64, 3)            0         ['input_2[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 32, 32, 16)           432       ['rescaling_1[0][0]']         
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 32, 32, 16)           64        ['Conv[0][0]']                
 lization)                                                                                  

In [44]:
model_mobilenet.compile(
    optimizer=AdamW(learning_rate=0.0001, weight_decay=0.0001),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [45]:
model_mobilenet.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/10


2024-01-30 11:36:01.459152: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 932216832 exceeds 10% of free system memory.


592/593 [============================>.] - ETA: 0s - loss: 0.7145 - accuracy: 0.5640

2024-01-30 11:36:37.645830: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 233078784 exceeds 10% of free system memory.


593/593 [==============================] - 39s 59ms/step - loss: 0.7144 - accuracy: 0.5640 - val_loss: 0.6580 - val_accuracy: 0.6031
Epoch 2/10
593/593 [==============================] - 32s 53ms/step - loss: 0.6637 - accuracy: 0.6069 - val_loss: 0.6418 - val_accuracy: 0.6472
Epoch 3/10
593/593 [==============================] - 28s 47ms/step - loss: 0.6455 - accuracy: 0.6258 - val_loss: 0.6299 - val_accuracy: 0.6520
Epoch 4/10
593/593 [==============================] - 26s 44ms/step - loss: 0.6341 - accuracy: 0.6414 - val_loss: 0.6165 - val_accuracy: 0.6662
Epoch 5/10
593/593 [==============================] - 25s 43ms/step - loss: 0.6243 - accuracy: 0.6500 - val_loss: 0.6049 - val_accuracy: 0.6771
Epoch 6/10
593/593 [==============================] - 26s 44ms/step - loss: 0.6193 - accuracy: 0.6581 - val_loss: 0.5953 - val_accuracy: 0.6894
Epoch 7/10
593/593 [==============================] - 27s 45ms/step - loss: 0.6106 - accuracy: 0.6677 - val_loss: 0.5906 - val_accuracy: 0.6944
Epo

## Labelled Faces in the Wild (LFW)
used to generate the adversary input

In [16]:
import os
lfw_root = 'data/lfw-deepfunneled'
all_lfw_names = os.listdir(lfw_root)

In [17]:
from keras.utils import load_img, img_to_array
from keras.applications.mobilenet import preprocess_input
import numpy as np

X = []

for name in all_lfw_names:
    dir_path = os.path.join(lfw_root, name)
    list_images_name = os.listdir(dir_path)
    
    for image_name in list_images_name:
        image_path = os.path.join(dir_path, image_name)
        rgb_image = load_img(image_path, target_size=(64, 64))
        x = img_to_array(rgb_image)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X.append(x)

X = np.asarray(X)

In [18]:
X[0].shape

(1, 64, 64, 3)